# VHF Ontology Explorer (CC-105)

Interactive exploration of VE ontologies using NetworkX visualization.

**Features:**
- Load and parse JSON-LD ontologies
- Build NetworkX graphs
- Interactive PyVis visualization
- Domain filtering (VE, CE, Agent)
- Path analysis and highlighting

In [ ]:
# Setup path to tools
import sys
from pathlib import Path

# Add tools directory to path
tools_path = Path('../tools').resolve()
if str(tools_path) not in sys.path:
    sys.path.insert(0, str(tools_path))

print(f"Tools path: {tools_path}")

In [ ]:
# Import VHF ontology tools
from ontology_loader import OntologyLoader, load_ontology
from graph_builder import OntologyGraphBuilder, build_ontology_graph, get_graph_stats
from visualiser import OntologyVisualiser, render_interactive

import networkx as nx
from IPython.display import HTML, display

print("VHF Ontology Tools loaded successfully!")

## 1. Load Ontology File

Select and load a JSON-LD ontology file.

In [ ]:
# List available ontology files
import glob

ontology_patterns = [
    '../PBS/ONTOLOGIES/**/*.json',
    '../../PF-Core-BAIV/PBS/ONTOLOGIES/**/*.json'
]

ontology_files = []
for pattern in ontology_patterns:
    ontology_files.extend(glob.glob(pattern, recursive=True))

print(f"Found {len(ontology_files)} ontology files:")
for i, f in enumerate(ontology_files[:10]):
    print(f"  {i}: {Path(f).name}")
if len(ontology_files) > 10:
    print(f"  ... and {len(ontology_files) - 10} more")

In [ ]:
# Load a specific ontology (change index or path as needed)
# Example: Load VSOM ontology from PF-Core-BAIV

ontology_path = None

# Try to find VSOM ontology
for f in ontology_files:
    if 'vsom' in f.lower():
        ontology_path = f
        break

# Fallback to first available
if not ontology_path and ontology_files:
    ontology_path = ontology_files[0]

if ontology_path:
    print(f"Loading: {ontology_path}")
    ontology = load_ontology(ontology_path)
    print(f"\nOntology: {ontology.name}")
    print(f"Version: {ontology.version}")
    print(f"Entities: {len(ontology.entities)}")
    print(f"Relationships: {len(ontology.relationships)}")
else:
    print("No ontology files found. Please specify a path manually.")

## 2. Explore Ontology Structure

In [ ]:
# List entities
print("=== ENTITIES ===")
for entity in ontology.entities[:15]:
    desc = entity.description[:60] + '...' if len(entity.description) > 60 else entity.description
    print(f"  {entity.label}: {desc}")

if len(ontology.entities) > 15:
    print(f"  ... and {len(ontology.entities) - 15} more")

In [ ]:
# List relationships
print("=== RELATIONSHIPS ===")
for rel in ontology.relationships[:15]:
    print(f"  {rel.source} --[{rel.label}]--> {rel.target} ({rel.cardinality})")

if len(ontology.relationships) > 15:
    print(f"  ... and {len(ontology.relationships) - 15} more")

In [ ]:
# List business rules
print("=== BUSINESS RULES ===")
for rule in ontology.business_rules[:10]:
    if isinstance(rule, dict):
        print(f"  {rule.get('id', 'rule')}: {rule.get('rule', rule)}")
    else:
        print(f"  {rule}")

## 3. Build NetworkX Graph

In [ ]:
# Build graph from ontology
builder = OntologyGraphBuilder()
G = builder.build_graph(ontology)

# Get statistics
stats = get_graph_stats(G)

print(f"=== GRAPH STATISTICS ===")
print(f"Nodes: {stats['nodes']}")
print(f"Edges: {stats['edges']}")
print(f"Density: {stats['density']:.4f}")
print(f"Is DAG: {stats['is_dag']}")
print(f"Components: {stats['components']}")
print(f"\nNode types: {stats['node_types']}")
print(f"Edge types: {stats['edge_types']}")

## 4. Interactive Visualization

In [ ]:
# Render interactive graph with PyVis
vis = OntologyVisualiser(height="600px")

# Generate HTML file
html_path = vis.render_pyvis(G, output_path='ontology_viz.html', physics=True)
print(f"Interactive visualization saved to: {html_path}")

# Display in notebook (if running in Jupyter)
try:
    from IPython.display import IFrame
    display(IFrame(src=html_path, width='100%', height='620px'))
except:
    print(f"Open {html_path} in a browser to view the interactive graph.")

## 5. Domain Filtering

In [ ]:
# Filter by domain
domains = ['VE', 'CE', 'Agent', 'Core', 'Framework']

for domain in domains:
    filtered = vis.filter_by_domain(G, domain, include_connected=False)
    if filtered.number_of_nodes() > 0:
        print(f"{domain}: {filtered.number_of_nodes()} nodes, {filtered.number_of_edges()} edges")

In [ ]:
# Visualize a specific domain (change as needed)
domain = 'VE'
filtered_graph = vis.filter_by_domain(G, domain, include_connected=True)

if filtered_graph.number_of_nodes() > 0:
    html_path = vis.render_pyvis(filtered_graph, output_path=f'{domain.lower()}_domain.html')
    print(f"{domain} domain visualization saved to: {html_path}")
    
    try:
        display(IFrame(src=html_path, width='100%', height='620px'))
    except:
        pass
else:
    print(f"No nodes found for domain: {domain}")

## 6. Path Analysis

In [ ]:
# Find all nodes (for path selection)
nodes = list(G.nodes())
print(f"Available nodes ({len(nodes)} total):")
for n in nodes[:20]:
    print(f"  {n}")
if len(nodes) > 20:
    print(f"  ... and {len(nodes) - 20} more")

In [ ]:
# Highlight path between two nodes (modify start/end as needed)
if len(nodes) >= 2:
    start = nodes[0]
    end = nodes[-1]
    
    print(f"Finding path from '{start}' to '{end}'...")
    
    try:
        path = nx.shortest_path(G, start, end)
        print(f"Path found: {' -> '.join(str(n) for n in path)}")
        
        # Highlight the path
        highlighted = vis.highlight_path(G, start, end)
        html_path = vis.render_pyvis(highlighted, output_path='path_highlighted.html')
        print(f"Highlighted path saved to: {html_path}")
        
    except nx.NetworkXNoPath:
        print(f"No path found between {start} and {end}")
else:
    print("Not enough nodes for path analysis")

## 7. Static Export

In [ ]:
# Export as static PNG image
try:
    png_path = vis.render_matplotlib(G, output_path='ontology_static.png', layout='spring')
    print(f"Static image saved to: {png_path}")
    
    # Display in notebook
    from IPython.display import Image
    display(Image(filename=png_path))
except Exception as e:
    print(f"Could not generate static image: {e}")

## 8. Graph Analysis

In [ ]:
# Centrality analysis
print("=== CENTRALITY ANALYSIS ===")

# Degree centrality (most connected nodes)
degree_centrality = nx.degree_centrality(G)
top_nodes = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 most connected nodes (degree centrality):")
for node, centrality in top_nodes:
    label = G.nodes[node].get('label', node)
    print(f"  {label}: {centrality:.4f}")

In [ ]:
# In/Out degree analysis
print("=== IN/OUT DEGREE ANALYSIS ===")

# Nodes with highest in-degree (most depended upon)
in_degree = dict(G.in_degree())
top_in = sorted(in_degree.items(), key=lambda x: x[1], reverse=True)[:5]

print("\nTop 5 most referenced nodes (in-degree):")
for node, degree in top_in:
    label = G.nodes[node].get('label', node)
    print(f"  {label}: {degree}")

# Nodes with highest out-degree (most dependencies)
out_degree = dict(G.out_degree())
top_out = sorted(out_degree.items(), key=lambda x: x[1], reverse=True)[:5]

print("\nTop 5 nodes with most outgoing edges (out-degree):")
for node, degree in top_out:
    label = G.nodes[node].get('label', node)
    print(f"  {label}: {degree}")

---

## Notes

- **Ontology files**: Place JSON-LD files in `PBS/ONTOLOGIES/` or point to PF-Core-BAIV
- **Interactive viz**: Open generated HTML files in browser for full interactivity
- **VE integration**: Use `ve_domain_graphs.py` for 8-layer business framework analysis
- **Agent context**: Use `AgentContextGraphBuilder` for agent-ontology relationships